# Client

Demo of a client interacting with a remote agent. 

You can interact with this via API directly

In [1]:
import requests

inputs = {"input": {"input": "what does eugene think of cats?"}}
response = requests.post("http://localhost:8000/invoke", json=inputs)

response.json()

{'output': {'output': 'Eugene thinks that cats like fish.'},
 'callback_events': [],
 'metadata': {'run_id': '5f91b070-ad79-46b9-95ae-761b050d30dc'}}

You can also interact with this via the RemoteRunnable interface (to use in other chains)

In [2]:
from langserve import RemoteRunnable

remote_runnable = RemoteRunnable("http://localhost:8000/")

Remote runnable has the same interface as local runnables

In [3]:
await remote_runnable.ainvoke({"input": "hi!"})

{'output': 'Hello! How can I assist you today?'}

In [4]:
remote_runnable.invoke({"input": "what does eugene think of cats?"})

{'output': 'Eugene thinks that cats like fish.'}

stream

In [7]:
async for chunk in remote_runnable.astream({"input": "what does eugene think of cats? Then tell me a story about that thought."}):
    print('--')
    print(chunk)

--
{'actions': [AgentActionMessageLog(tool='get_eugene_thoughts', tool_input={'query': 'cats'}, log="\nInvoking: `get_eugene_thoughts` with `{'query': 'cats'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_eugene_thoughts', 'arguments': '{\n"query": "cats"\n}'}})])], 'messages': [AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_eugene_thoughts', 'arguments': '{\n"query": "cats"\n}'}})]}
--
{'steps': [{'action': AgentActionMessageLog(tool='get_eugene_thoughts', tool_input={'query': 'cats'}, log="\nInvoking: `get_eugene_thoughts` with `{'query': 'cats'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_eugene_thoughts', 'arguments': '{\n"query": "cats"\n}'}})]), 'observation': [Document(page_content='cats like fish'), Document(page_content='dogs like sticks')]}], 'messages': [FunctionMessage(content="[Document(page_content='cats like fish'), Document(page_content='dogs like s

In [6]:
async for chunk in remote_runnable.astream_log({"input": "what does eugene think of cats?"}):
    print('--')
    print(chunk)

--
RunLogPatch({'op': 'replace',
  'path': '',
  'value': {'final_output': None,
            'id': '29e1bb83-c4f2-47b6-a9fd-e34f4584d261',
            'logs': {},
            'streamed_output': []}})
--
RunLogPatch({'op': 'add',
  'path': '/logs/RunnableSequence',
  'value': {'end_time': None,
            'final_output': None,
            'id': '5f13d17e-f653-47b5-b47c-550a3ad7ee91',
            'metadata': {'__langserve_endpoint': 'stream_log',
                         '__langserve_version': '0.0.37',
                         '__useragent': 'python-httpx/0.25.2'},
            'name': 'RunnableSequence',
            'start_time': '2024-01-05T18:20:39.558+00:00',
            'streamed_output': [],
            'streamed_output_str': [],
            'tags': [],
            'type': 'chain'}})
--
RunLogPatch({'op': 'add',
  'path': '/logs/RunnableParallel<input,agent_scratchpad>',
  'value': {'end_time': None,
            'final_output': None,
            'id': '79cc1c10-4226-447f-bc87-540e